In [ ]:
import requests
from IPython.display import Image, display


def fetch_books(query):
    url = f'https://openlibrary.org/search.json?q={query}'
    response = requests.get(url)
    data = response.json()
    books = data.get('docs', [])
    return books[:1] 


def fetch_similar_books_by_subject(subject):
    subject = subject.replace(' ', '_').lower() 
    url = f'https://openlibrary.org/subjects/{subject}.json?limit=10'
    try:
        response = requests.get(url)
        response.raise_for_status()  
        data = response.json()
        similar_books = data.get('works', [])
    except (requests.exceptions.RequestException, ValueError) as e:
        print(f"Error fetching similar books by subject '{subject}': {e}")
        similar_books = []
    return similar_books

def fetch_trending_books():
    url = 'https://openlibrary.org/trending/monthly.json?limit=10'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        trending_books = data.get('works', [])
    except (requests.exceptions.RequestException, ValueError) as e:
        print(f"Error fetching trending books: {e}")
        trending_books = []
    return trending_books

def display_books(books, recommendation_type):
    for book in books:
        title = book.get('title')
        author_name = ', '.join(book.get('author_name', []))
        cover_id = book.get('cover_i')
        
        print(f"Title: {title}")
        print(f"Author: {author_name}")
        
        if cover_id:
            cover_url = f'http://covers.openlibrary.org/b/id/{cover_id}-M.jpg'
            display(Image(cover_url))
        
        print('\n' + '='*50 + '\n')

        if recommendation_type == 'author':
            recommended_book, reason = get_recommendation_by_author(book)
        elif recommendation_type == 'genre':
            recommended_book, reason = get_recommendation_by_genre(book)
        else:
            recommended_book, reason = (None, None)

        if not recommended_book:
            recommended_book, reason = get_fallback_recommendation()

        if recommended_book:
            rec_title = recommended_book.get('title')
            rec_author_name = ', '.join(recommended_book.get('author_name', []))
            rec_cover_id = recommended_book.get('cover_i')
            
            print(f"Recommended Book: {rec_title}")
            print(f"Author: {rec_author_name}")
            print(f"Reason: {reason}")
            
            if rec_cover_id:
                rec_cover_url = f'http://covers.openlibrary.org/b/id/{rec_cover_id}-M.jpg'
                display(Image(rec_cover_url))
            
            print('\n' + '='*50 + '\n')

def get_recommendation_by_author(book):
    author_name = ', '.join(book.get('author_name', []))
    original_work_id = book.get('key')
    author_books = fetch_books_by_author(author_name, original_work_id)
    author_books = [b for b in author_books if b.get('key') != original_work_id and b.get('title') != book.get('title')]
    if author_books:
        recommended_book = author_books[0]
        reason = f"This book is recommended because it is also written by {author_name}."
        return recommended_book, reason
    return None, None

def get_recommendation_by_genre(book):
    subjects = book.get('subject', [])
    for subject in subjects[:5]:  
        similar_books = fetch_similar_books_by_subject(subject)
        similar_books = [b for b in similar_books if b.get('key') != book.get('key')]
        if similar_books:
            recommended_book = similar_books[0]
            reason = f"This book is recommended because it shares similar subjects: {', '.join(subjects[:5])}."
            return recommended_book, reason
    return None, None

def get_fallback_recommendation():
    trending_books = fetch_trending_books()
    if trending_books:
        recommended_book = trending_books[0]
        reason = "This book is recommended because it's currently trending."
        return recommended_book, reason
    return None, "No recommendations available."


def fetch_books_by_author(author_name, original_work_id):
    url = f'https://openlibrary.org/search.json?author={author_name.replace(" ", "+")}'
    response = requests.get(url)
    data = response.json()
    books = data.get('docs', [])
    return [book for book in books if book.get('key') != original_work_id]


query = input("Enter a book title or author: ")
books = fetch_books(query)
if books:
    recommendation_type = input("Do you want recommendations based on 'author' or 'genre'? ")
    display_books(books, recommendation_type)
else:
    print("No such book in the database.")
